기존 토크나이저에서 새로운 토크나이저 학습  
- 토크나이저 학습은 주어진 말뭉치에 대해 어떤 하위 단어(subword)를 선택하는 것이 가장 좋은지 식별하려는 통계적 프로세스
- 이를 선택하는데 사용되는 정확한 규칙은 토큰화 알고리즘에 따라 다름. 이 과정은 결정론적(deterministic)
- 동일한 말뭉치에서 동일한 알고리즘으로 학습하면 항상 동일한 결과를 얻을 수 있음

말뭉치 모으기  
- Transformers에는 기존에 존재하는 것들과 동일한 특성을 가진 새로운 토크나이저를 학습하는데 사용할 수 있는 AutoTokenizer.train_new_from_iterator()가 있음
- 실제로 실행해 보기 위해 GPT-2를 처음부터 영어가 아닌 다른 언어로 학습하고 싶다고 가정
  - 첫 번째 작업은 해당 언어로 표현된 대규모의 데이터를 수집하여 학습 말뭉치로 구성하는 것
  - 특정 언어가 아니라 특수한 영어 텍스트로 볼 수 있는 Python 소스코드 집합 사용
- Datasets 라이브러리를 통해 Python 소스코드를 모으는데 load_dataset() 함수를 사용하여 CodeSearchNet 데이터셋을 다운로드하고 캐시함
- 이 데이터셋은 CodeSearchNet 챌린지를 위해 생성되어 여러 프로그래밍 언어로 된 GitHub의 오픈소스 라이브러리에서 수백만 개의 함수를 포함하고 있음

In [1]:
from datasets import load_dataset

raw_datasets = load_dataset('code_search_net', 'python')

# data split & show columns
print(raw_datasets['train'])

# 이 데이터셋이 코드에서 독스트링(docstrings)을 분리하고 있고
# 이 코드와 독스트링에 대한 토큰화가 필요함을 알 수 있음 
# 토크나이저를 학습하기 위해 whole_func_string 열만을 사용함
# train split을 인덱싱하여 함수들 중 하나의 예시 확인
print(raw_datasets['train'][123456]['whole_func_string'])

Found cached dataset code_search_net (/root/.cache/huggingface/datasets/code_search_net/python/1.0.0/8f2524e6b62f65af5f5d65c53715c654db7b08dc93e0b7bcce2ab2f286a75be1)


  0%|          | 0/3 [00:00<?, ?it/s]

Dataset({
    features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
    num_rows: 412178
})
def is_solved(self):
        """
        Check if Cube's F2L is solved.
        """
        if self.cube.D == [[Square(self.cube["D"].colour)] * 3] * 3:
            for face in "LFRB":
                if self.cube.get_face(face)[1:] != [[Square(self.cube[face].colour)] * 3] * 2:
                    return False
            return True
        return False


- 가장 먼저 데이터셋을 텍스트 리스트의 이터레이터(iterator)로 변환하는 것
  - 예를 들어, 텍스트 리스트의 리스트로 구성할 수 있음
  - 텍스트 리스트를 사용하면 개별 텍스트를 하나씩 처리하는 대신 텍스트 배치(batches)에 대한 학습을 통해서 토크나이저가 더 빨라질 수 있음
  - 모든 것을 한 번에 메모리에 로딩하지 않으려면 이 리스트를 이터레이터(iterator)로 변환되어야 함
  - 말뭉치의 규모가 크다면 Datasets는 RAM에 로드하지 않고 데이터셋의 요소를 디스크에 저장한다는 사실을 활용할 수 있음

In [2]:
# 제너레이터 이용
def get_training_corpus():
    return (
        raw_datasets['train'][i: i + 1000]['whole_func_string']
        for i in range(0, len(raw_datasets['train']), 1000))
training_corpus = get_training_corpus()

# yield 사용하여 for loop 내에서 제너레이터 정의 (list comprehension)
# def get_training_corpus():
#     dataset = raw_datasets['train']
#     for start_idx in range(0, len(datset, 1000):
#         samples = dataset[start_idx : start_idx + 1000]
#         yield samples['whole_func_string']

새로운 토크나이저 학습

In [3]:
# 학습을 위해서 GPT-2 모델과 일치시키려는 토크나이저를 로드
from transformers import AutoTokenizer

old_tokenizer = AutoTokenizer.from_pretrained('gpt2')

- 새로운 토크나이저를 학습하지만 완전히 처음부터 시작하지 않도록 하는 것이 좋음
- 이렇게 하면 토큰화 알고리즘이나 사용하려는 특수 토큰(special tokens)에 대해 아무 것도 신경쓰거나 지정할 필요가 없음
- 새로운 토크나이저는 GPT-2와 동일할 것이며, 우리 말뭉치를 이용한 학습을 통해 vocabulary만 변경됨

In [4]:
# 토크나이저가 example function을 처리하는 방법
example = '''def add_numbers(a, b):
    """Add the two numbers `a` and `b`."""
    return a + b'''
tokens = old_tokenizer.tokenize(example)
print(tokens)

['def', 'Ġadd', '_', 'n', 'umbers', '(', 'a', ',', 'Ġb', '):', 'Ċ', 'Ġ', 'Ġ', 'Ġ', 'Ġ"""', 'Add', 'Ġthe', 'Ġtwo', 'Ġnumbers', 'Ġ`', 'a', '`', 'Ġand', 'Ġ`', 'b', '`', '."', '""', 'Ċ', 'Ġ', 'Ġ', 'Ġ', 'Ġreturn', 'Ġa', 'Ġ+', 'Ġb']


- 이 토크나이저는 각각 공백과 줄바꿈을 나타내는 Ċ 및 Ġ와 같은 특수 기호를 포함하고 있음
- 여러 개의 공백이 나타날 때 토크나이저는 이를 그룹화하여 하나의 토큰으로 표현할 수도 있는데, 여기서는 각 공백을 개별 토큰으로 표현하고 있음
  - 소스코드에서 4개 또는 8개의 공백 그룹이 나타나는 것은 매우 일반적임

In [5]:
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 52000)

In [6]:
# 대부분의 트랜스포머 모델에는 사용 가능한 빠른 토크나이저가 있음
# AutoTokenizer API는 사용 가능한 경우 항상 빠른 토크나이저를 선택함
# 새로운 토크나이저 사용
tokens = tokenizer.tokenize(example)
print(tokens)
print(len(tokens), len(old_tokenizer.tokenize(example)))

['def', 'Ġadd', '_', 'numbers', '(', 'a', ',', 'Ġb', '):', 'ĊĠĠĠ', 'Ġ"""', 'Add', 'Ġthe', 'Ġtwo', 'Ġnumbers', 'Ġ`', 'a', '`', 'Ġand', 'Ġ`', 'b', '`."""', 'ĊĠĠĠ', 'Ġreturn', 'Ġa', 'Ġ+', 'Ġb']
27 36


In [7]:
# 다른 예
example = """class LinearLayer():
    def __init__(self, input_size, output_size):
        self.weight = torch.randn(input_size, output_size)
        self.bias = torch.zeros(output_size)

    def __call__(self, x):
        return x @ self.weights + self.bias
    """
print(tokenizer.tokenize(example))

['class', 'ĠLinear', 'Layer', '():', 'ĊĠĠĠ', 'Ġdef', 'Ġ__', 'init', '__(', 'self', ',', 'Ġinput', '_', 'size', ',', 'Ġoutput', '_', 'size', '):', 'ĊĠĠĠĠĠĠĠ', 'Ġself', '.', 'weight', 'Ġ=', 'Ġtorch', '.', 'randn', '(', 'input', '_', 'size', ',', 'Ġoutput', '_', 'size', ')', 'ĊĠĠĠĠĠĠĠ', 'Ġself', '.', 'bias', 'Ġ=', 'Ġtorch', '.', 'zeros', '(', 'output', '_', 'size', ')', 'ĊĊĠĠĠ', 'Ġdef', 'Ġ__', 'call', '__(', 'self', ',', 'Ġx', '):', 'ĊĠĠĠĠĠĠĠ', 'Ġreturn', 'Ġx', 'Ġ@', 'Ġself', '.', 'weights', 'Ġ+', 'Ġself', '.', 'bias', 'ĊĠĠĠĠ']


In [8]:
tokenizer.save_pretrained('code-search-net-tokenizer')
tokenizer.push_to_hub('code-search-net-tokenizer', use_temp_dir=True)

CommitInfo(commit_url='https://huggingface.co/groov/code-search-net-tokenizer/commit/2fef017d9f57c81009346677374bdfeb1210155c', commit_message='Upload tokenizer', commit_description='', oid='2fef017d9f57c81009346677374bdfeb1210155c', pr_url=None, pr_revision=None, pr_num=None)

"빠른(fast)" 토크나이저의 특별한 능력  
- 느린(slow) 토크나이저는 Transformers 라이브러리 내부에서 Python으로 작성된 것
- 빠른 버전은 Rust로 작성되어 Tokenizers에서 제공하는 것
- 단일 문장을 토큰화할 때 동일한 토크나이저의 느린 버전과 빠른 버전 간의 속도 차이가 항상 나는 것은 아님. 빠른 버전은 실제로 더 느릴 수 있음. 많은 텍스트를 동시에 토큰화할 때 차이를 알 수 있음

배치 인코딩 (Batch encoding)  
- 토크나이저의 출력은 단순한 Python 딕셔너리가 아닌 BatchEncoding 객체
- 이것은 딕셔너리의 하위 클래스이지만(이것이 이전에 문제없이 결과를 색인화할 수 있었던 이유) 빠른 토크나이저에서 주로 사용하는 추가 메서드가 있음
- 병렬화(parallelization) 기능 외에도 빠른 토크나이저의 주요 기능은 최종 토큰이 원본 텍스트에서 어디에 위치하는지 범위(span)를 항상 추적한다는 것
  - 이를 오프셋 매핑(offset mapping) 이라고 함
  - 이것은 차례대로 각 단어를 생성된 토큰에 매핑하거나 원본 텍스트의 각 문자를 내부 토큰에 매핑하거나 그 반대로 매핑하는 것과 같은 기능

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
example = 'My name is Sylvain and I work at Hugging Face in Brooklyn.'
encoding = tokenizer(example)
print(type(encoding))

<class 'transformers.tokenization_utils_base.BatchEncoding'>


In [10]:
# 토크나이저의 출력에서 BatchEncoding 객체를 얻음
# AutoTokenizer 클래스는 기본적으로 빠른 토크나이저를 선택하므로 
# BatchEncoding 객체가 제공하는 추가 메서드를 사용할 수 있음
# 토크나이저의 is_fast 속성을 통해 빠른지 느린지 확인 할 수 있음
print(tokenizer.is_fast)

# 토큰 아이디를 다시 토큰으로 변환하지 않고도 토큰에 엑세스할 수 있음
print(encoding.tokens())

# 이 경우 인덱스 5의 토큰은 ##yl이며 이는 원래 문장에서 "Sylvain"이라는 단어의 일부.
# word_ids() 메서드를 사용하여 각 토큰이 유래된 해당 단어의 인덱스를 가져올 수 있음
print(encoding.word_ids())

True
['[CLS]', 'My', 'name', 'is', 'S', '##yl', '##va', '##in', 'and', 'I', 'work', 'at', 'Hu', '##gging', 'Face', 'in', 'Brooklyn', '.', '[SEP]']
[None, 0, 1, 2, 3, 3, 3, 3, 4, 5, 6, 7, 8, 8, 9, 10, 11, 12, None]


- 토크나이저의 특수 토큰 [CLS] 및 [SEP]가 None으로 매핑된 다음, 개별 토큰들이 해당 토큰이 유래한 단어에 매핑되는 것을 볼 수 있음
- 이 메서드는 두 개의 토큰이 같은 단어에 있는지 아니면 토큰이 단어의 시작 부분에 있는지 확인하는데 특히 유용함
  - 이를 위해 ## 접두사를 사용할 수 있지만 이는 BERT와 같은 유형의 토크나이저에서만 작동함
  - 이 방법은 속도가 빠른 모든 유형의 토크나이저에서 유효함
- 단어가 무엇인지에 대한 개념
  - 예를 들어, "I'll"은 하나의 단어일까? 아니면 두개일까? 이는 실제로 토크나이저와 적용되는 사전 토큰화(pre-tokenization) 작업에 따라 다름
  - 일부 토크나이저는 공백 기준으로 분할하므로 이를 한 단어로 간주함. 또 다른 토크나이저들은 공백 위에 구두점을 사용하므로 두 단어로 간주함
- 토큰을 가져온 문장에 해당 토큰을 매핑하는데 사용할 수 있는 sentence_ids() 메서드가 있음(이 경우 토크나이저에서 반환된 token_type_ids가 동일한 정보를 제공할 수 있음)
- word_to_chars(), token_to_chars(), char_to_word(), char_to_token() 메서드를 통해 모든 단어 또는 토큰을 원본 텍스트의 문자에 매핑할 수 있으며 그 반대로도 매핑할 수 있음

In [11]:
# 예를 들어 word_ids() 메서드는 ##yl이 인덱스 3에 있는 단어의 일부라고 알려줬지만 
# 정확히 문장 내에서 어떤 단어에 해당하는 것인 지는 다음과 같이 알 수 있음
start, end = encoding.word_to_chars(3)
# 빠른 토크나이저가 offset목록에서 각 토큰이 가져온 텍스트 범위(span)를 추적한다는 
# 사실에 기반하여 구동됨
example[start:end]

'Sylvain'

In [12]:
# 파이프라인으로 기본 실행 결과 도출하기
from transformers import pipeline

token_classifier = pipeline('token-classification')
print(token_classifier('My name is Sylvain and I work at Hugging Face in Brooklyn.'))

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'entity': 'I-PER', 'score': 0.99938285, 'index': 4, 'word': 'S', 'start': 11, 'end': 12}, {'entity': 'I-PER', 'score': 0.99815494, 'index': 5, 'word': '##yl', 'start': 12, 'end': 14}, {'entity': 'I-PER', 'score': 0.9959072, 'index': 6, 'word': '##va', 'start': 14, 'end': 16}, {'entity': 'I-PER', 'score': 0.99923277, 'index': 7, 'word': '##in', 'start': 16, 'end': 18}, {'entity': 'I-ORG', 'score': 0.9738931, 'index': 12, 'word': 'Hu', 'start': 33, 'end': 35}, {'entity': 'I-ORG', 'score': 0.976115, 'index': 13, 'word': '##gging', 'start': 35, 'end': 40}, {'entity': 'I-ORG', 'score': 0.9887976, 'index': 14, 'word': 'Face', 'start': 41, 'end': 45}, {'entity': 'I-LOC', 'score': 0.9932106, 'index': 16, 'word': 'Brooklyn', 'start': 49, 'end': 57}]


- 모델은 "Sylvain"에서 분리된 각 토큰들을 모두 사람(person)
- "Hugging Face"에서 분리된 각 토큰들을 모두 조직(organization)
- "Brooklyn" 토큰을 위치(location)로 식별

In [13]:
# 파이프라인에 동일한 엔터티에 해당하는 토큰을 그룹화하도록 요청할 수도 있음
from transformers import pipeline

token_classifier = pipeline(
    'token-classification', aggregation_strategy='simple')
token_classifier('My name is Sylvain and I work at Hugging Face in Brooklyn.')

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'entity_group': 'PER',
  'score': 0.9981694,
  'word': 'Sylvain',
  'start': 11,
  'end': 18},
 {'entity_group': 'ORG',
  'score': 0.9796019,
  'word': 'Hugging Face',
  'start': 33,
  'end': 45},
 {'entity_group': 'LOC',
  'score': 0.9932106,
  'word': 'Brooklyn',
  'start': 49,
  'end': 57}]

- aggregation_strategy를 위와 같이 지정하면 토큰들이 하나로 합쳐진 엔터티에 대해 새롭게 계산된 스코어를 제시함
  - "simple"의 경우 스코어는 해당 개체명 내의 각 토큰에 대한 스코어의 평균
- "Sylvain"의 스코어는 이전 예에서 S, ##yl, ##va 및 ##in 토큰에 대해 계산된 스코어의 평균 사용 가능한 다른 지정자는 다음과 같음
  - "first" : 각 개체명의 스코어는 해당 개체명의 첫 번째 토큰의 스코어(따라서 "Sylvain"의 경우 토큰 S의 점수인 0.993828이 됨)
  - "max" : 각 엔터티의 스코어는 해당 엔터티내의 토큰들 중의 최대값 스코어("Hugging Face"의 경우 "Face"의 점수는 0.98879766이 됨).
  - "average" : 각 항목의 스코어는 해당 항목을 구성하는 단어 스코어의 평균(따라서 "Sylvain"의 경우 "simple" 지정자와 차이가 없지만 "Hugging Face"의 점수는 0.9819이며 "Hugging"은 0.975이고 "Face"는 0.98879).

입력(inputs)에서 예측(predictions)까지  
- 먼저 입력을 토큰화하고 모델을 통해 전달해야 함
- AutoXxx 클래스를 사용하여 토크나이저와 모델을 인스턴스화한 후에 이를 예제에서 사용함

In [14]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

model_checkpoint = 'dbmdz/bert-large-cased-finetuned-conll03-english'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint)

example = 'My name is Sylvain and I work at Hugging Face in Brooklyn.'
inputs = tokenizer(example, return_tensors='pt')
outputs = model(**inputs)

# 입력 시퀀스의 각 토큰에 대해 하나의 logits 세트를 얻음
print(inputs['input_ids'].shape)
print(outputs.logits.shape)

# 19개의 토큰으로 구성된 1개의 시퀀스가 있는 batch가 있고 9개의 다른 레이블이 
# 존재하므로 shape은 1 x 19 x 9.
# text-classification 파이프라인과 마찬가지로 softmax 함수를 사용하여 해당 logits을 
# 확률로 변환하고 argmax를 사용하여 예측 결과를 얻을 수 있음
# (softmax는 순서를 변경하지 않기 때문에 logits에 대해서 argmax를 취할 수 있음)
import torch

probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)[0].tolist()
predictions = outputs.logits.argmax(dim=-1)[0].tolist()
print(probabilities)
print(predictions)

# 예측결과 확인하는데 사용하는 레이블에 대한 인덱스 매핑이 포함되어 있음
print(model.config.id2label)

torch.Size([1, 19])
torch.Size([1, 19, 9])
[[0.9994322657585144, 1.6470316040795296e-05, 3.426706462050788e-05, 1.6042342394939624e-05, 8.250699465861544e-05, 2.1382315026130527e-05, 0.00015649119450245053, 1.965213414223399e-05, 0.0002208924270235002], [0.9989632368087769, 1.851575871114619e-05, 5.2404582675080746e-05, 1.2534721463453025e-05, 0.000434736255556345, 3.087430013692938e-05, 0.0003146875533275306, 2.786070399452001e-05, 0.00014510867185890675], [0.9997084736824036, 8.308143151225522e-06, 2.8745698728016578e-05, 5.650369985232828e-06, 8.694890857441351e-05, 9.783487257664092e-06, 6.786159065086395e-05, 1.1794004421972204e-05, 7.241915591293946e-05], [0.9998351335525513, 5.645526925945887e-06, 1.3955141184851527e-05, 4.3133695726282895e-06, 4.0176873881137e-05, 8.123063707898837e-06, 5.648491423926316e-05, 8.991635695565492e-06, 2.7239091650699265e-05], [0.00018333387561142445, 2.5156617994070984e-05, 4.846198862651363e-05, 1.4900553651386872e-05, 0.9993828535079956, 1.99977

- O는 개체명에 포함되지 않는 토큰에 대한 레이블("outside"를 나타냄)이고 각 개체명 유형, 기타(miscellaneous), 인명(person), 기관명(organization), 지명(location) 각각에 대해 두 개의 레이블이 있음
- 레이블 B-XXX는 토큰이 개체명 XXX의 시작 부분에 있음을 나타내고 레이블 I-XXX는 토큰이 개체명 XXX의 내부에 있음을 나타냄
  - 예를 들어 이전 예시에서 우리는 모델이 토큰 "S"를 B-PER(인명 개체명의 시작)으로 분류하고 "##yl", "##va" 및 "##in" 토큰을 I-PER(인명 개체명의 내부)로 분류하기를 기대했을 수도 있음
  - 그런데 위 결과에서 보는 바와 같이 4개 토큰 모두에 I-PER이라는 레이블을 부여했기 때문에 예측에 오류가 있다고 생각할 수 있지만 그렇지 않을 수도 있음
- 실제로 이런 B- 및 I- 레이블 표기 방식에는 IOB1 및 IOB2의 두 가지 형식이 있음
  - IOB2 형식은 우리가 도입한 형식인 반면 IOB1 형식에서 B-로 시작하는 레이블은 동일한 유형의 인접한 두 엔터티를 구분하는 데만 사용됨
  - 우리가 사용하는 모델은 IOB1 형식을 사용하는 데이터셋에서 미세 조정되었으므로 "S" 토큰에 레이블 I-PER을 할당함

In [15]:
# O로 분류되지 않은 각 코튼의 점수와 레이블만 가져올 수 있음
results = []
tokens = inputs.tokens()

for idx, pred in enumerate(predictions):
    label = model.config.id2label[pred]
    if label != 'O':
        results.append(
            {'entity': label, 'score': probabilities[idx][pred], 
             'word': tokens[idx]})

# 파이프라인은 또한 원본 문장에서 각 엔터티의 시작과 끝에 대한 정보를 제공함
# 여기에서 오프셋 매핑(offset mapping)이 작동함
print(results)

# 오프셋(offset)을 얻으려면 입력에 토크나이저를 적용할 때 
# return_offsets_mapping=True를 설정하면 됨
inputs_with_offsets = tokenizer(example, return_offsets_mapping=True)
print(inputs_with_offsets['offset_mapping'])

# 각 튜플은 각 토큰에 해당하는 텍스트 범위이며 (0, 0)은 특수 토큰용으로 예약되어 있음. 
# 인덱스 5의 토큰이 "##yl"이고 오프셋이 (12, 14)로 지정되어 있음
print(example[12:14]) # '##' 없이 적절한 텍스트 범위를 얻음

results = []
inputs_with_offsets = tokenizer(example, return_offsets_mapping=True)
tokens = inputs_with_offsets.tokens()
offsets = inputs_with_offsets['offset_mapping']

for idx, pred in enumerate(predictions):
    label = model.config.id2label[pred]
    if label != 'O':
        start, end = offsets[idx]
        results.append(
            {'entity': label, 'score': probabilities[idx][pred],
             'word': tokens[idx], 'start': start, 'end': end})

print(results)

[{'entity': 'I-PER', 'score': 0.9993828535079956, 'word': 'S'}, {'entity': 'I-PER', 'score': 0.9981549382209778, 'word': '##yl'}, {'entity': 'I-PER', 'score': 0.995907187461853, 'word': '##va'}, {'entity': 'I-PER', 'score': 0.9992327690124512, 'word': '##in'}, {'entity': 'I-ORG', 'score': 0.9738931059837341, 'word': 'Hu'}, {'entity': 'I-ORG', 'score': 0.9761149883270264, 'word': '##gging'}, {'entity': 'I-ORG', 'score': 0.9887976050376892, 'word': 'Face'}, {'entity': 'I-LOC', 'score': 0.9932106137275696, 'word': 'Brooklyn'}]
[(0, 0), (0, 2), (3, 7), (8, 10), (11, 12), (12, 14), (14, 16), (16, 18), (19, 22), (23, 24), (25, 29), (30, 32), (33, 35), (35, 40), (41, 45), (46, 48), (49, 57), (57, 58), (0, 0)]
yl
[{'entity': 'I-PER', 'score': 0.9993828535079956, 'word': 'S', 'start': 11, 'end': 12}, {'entity': 'I-PER', 'score': 0.9981549382209778, 'word': '##yl', 'start': 12, 'end': 14}, {'entity': 'I-PER', 'score': 0.995907187461853, 'word': '##va', 'start': 14, 'end': 16}, {'entity': 'I-PER'

엔터티 그룹화  
- Hu, ##gging 및 Face 토큰을 하나로 그룹화하려는 경우 처음 두 토큰은 ## 없이 합쳐야 하고 Face는 ##로 시작하지 않으므로 앞에 공백을 추가하여 결합해야 한다는 특수 규칙을 만들어야 함
- 그러나 이러한 규칙들은 특정 유형의 토크나이저에서만 작동함
  - SentencePiece, Byte-Pair-Encoding 토크나이저에서는 또다른 규칙 집합을 작성해야 함
- 오프셋을 사용하면 모든 사용자 정의 코드가 사라짐. 단지 첫 번째 토큰으로 시작하고 마지막 토큰으로 끝나는 원본 텍스트의 범위를 사용할 수 있음
- Hu, ##gging 및 Face 토큰의 경우 합치기 위해 33번째 문자(Hu의 시작 부분)에서 시작하여 45번째 문자(Face의 끝 부분) 앞까지 슬라이싱을 해주면 됨

In [16]:
# 특정 엔터티에 포함된 토큰들을 그룹화하는 동안 예측 결과를 후처리하는 코드를 
# 작성하기 위해 B-XXX 또는 I-XXX로 레이블이 지정될 수 있는 첫 번째 엔터티를 
# 제외하고 연속적이고 I-XXX로 레이블이 지정된 엔터티를 함께 그룹화함.
# O, 새로운 유형의 엔터티 또는 동일한 유형의 엔터티가 시작되고 있음을 알리는 
# B-XXX를 받으면 엔터티 그룹화를 중지함

import numpy as np

results = []
inputs_with_offsets = tokenizer(example, return_offsets_mapping=True)
tokens = inputs_with_offsets.tokens()
offsets = inputs_with_offsets['offset_mapping']

idx = 0
while idx < len(predictions):
    pred = predictions[idx]
    label = model.config.id2label[pred]
    if label != 'O':
        # remove the B- or I-
        label = label[2:]
        start, _ = offsets[idx]
        
        # grab all the tokens labeled with I-label
        all_scores = []
        while (
            idx < len(predictions)
            and model.config.id2label[predictions[idx]] == f'I-{label}'):
            all_scores.append(probabilities[idx][pred])
            _, end = offsets[idx]
            idx += 1
            
        score = np.mean(all_scores).item()
        word = example[start:end]
        results.append(
            {'entity_group': label, 'score': score, 'word': word,
            'start': start, 'end': end,})
    idx += 1

print(results)

[{'entity_group': 'PER', 'score': 0.9981694370508194, 'word': 'Sylvain', 'start': 11, 'end': 18}, {'entity_group': 'ORG', 'score': 0.9796018997828165, 'word': 'Hugging Face', 'start': 33, 'end': 45}, {'entity_group': 'LOC', 'score': 0.9932106137275696, 'word': 'Brooklyn', 'start': 49, 'end': 57}]


정규화(Normalization) 및 사전 토큰화(Pre-tokenization)  
- 일반적으로 사용되는 subwword 토큰화 알고리즘 : Byte-Pair Encoding[BPE], WordPiece, Unigram
- 텍스트를 하위 토큰(subtokens)으로 분할하기 전에(모델에 따라) 토크나이저는 정규화(normalization) 및 사전 토큰화(pre-tokenization) 두 단계를 수행함

정규화(Normalization)  
- 정규화 단계에는 불필요한 공백 제거, 소문자 변환(lowercasing) 및 악센트 제거 등과 같은 일반적인 정제 작업이 포함됨
- NFC 또는 NFKC와 같은 유니코드 정규화(Unicode normalization) 작업과 거의 동일한 작업이 이 과정에서 수행됨

In [17]:
# Transformers의 tokenizer는 Tokenizers 라이브러리에서 하부 토크나이저에 대한 
# 액세스를 제공하는 backend_tokenizer라는 속성이 있음
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
print(type(tokenizer.backend_tokenizer))

# 토크나이저 객체의 normalizer 속성에는 normalize_str() 메서드가 있음. 
# 이 메서드는 정규화가 수행되는 방식을 확인하는데 사용할 수 있음.
# 이 예에서는 bert-base-uncased 체크포인트를 선택했기 때문에 정규화 과정에서 
# 소문자화(lowercasing)를 수행하고 악센트를 제거했음.
print(tokenizer.backend_tokenizer.normalizer.normalize_str("Héllò hôw are ü?"))

<class 'tokenizers.Tokenizer'>
hello how are u?


사전토큰화(Pre-tokenization)  
- 토크나이저는 원시 텍스트만으로는 학습될 수 없음. 대신에 먼저 텍스트를 단어와 같은 작은 개체들로 분할해야 함. 여기서 사전 토큰화(pre-tokenization) 단계가 실행됨
- 단어 기반 토크나이저는 원시 텍스트를 단순히 공백과 구두점을 기준으로 단어로 분할할 수 있음
  - 이 단어들은 토크나이저가 학습 과정에서 학습할 수 있는 하위 토큰(subtokens)의 경계가 됨

In [18]:
# 빠른 토크나이저가 사전 토큰화를 수행하는 과정을 보려면 tokenizer 객체의 
# pre_tokenizer 속성이 가진 pre_tokenize_str() 메서드를 사용할 수 있음
print(tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(
    'Hello, how are you?'))

# 토크나이저가 오프셋을 유지할 때 오프셋 매핑을 제공함.
# 두 개의 공백("are"와 "you" 사이에 있는)을 무시하고 하나의 공백으로 바꾸지만
# "are"와 "you" 사이의 오프셋 점프(14에서 16)는 계속 유지함.
# BERT 토크나이저 사전 토큰화에는 공백(whitespace)과 구두점(puntuation) 분할이 포함됨.

# GPT-2 토크나이저는 이 단계에서 다른 규칙을 수행할 수 있음
tokenizer = AutoTokenizer.from_pretrained('gpt2')
print(tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(
    'Hello, how are you?'))

# 공백과 구두점에서도 분할되지만 공백은 없애지 않고 Ġ 기호로 대체하므로 토큰을 
# 디코딩하면 원래 공백을 복구할 수 있음.
# 또한 BERT 토크나이저와 달리 이 토크나이저는 이중 공백을 무시하지 않음
# T5 토크나이저의 예
tokenizer = AutoTokenizer.from_pretrained("t5-small")
print(tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(
    'Hello, how are  you?'))

# GPT-2 토크나이저와 같이 T5 토크나이저는 공백을 유지하고 특정 토큰(_)으로 
# 대체하지만 구두점이 아닌 공백에서만 토큰을 분할함
# 문장 시작 부분("Hello" 앞부분)에 공백을 추가하고 "are"와 "you" 사이의 이중 공백 무시

[('Hello', (0, 5)), (',', (5, 6)), ('how', (7, 10)), ('are', (11, 14)), ('you', (15, 18)), ('?', (18, 19))]
[('Hello', (0, 5)), (',', (5, 6)), ('Ġhow', (6, 10)), ('Ġare', (10, 14)), ('Ġyou', (14, 18)), ('?', (18, 19))]
[('▁Hello,', (0, 6)), ('▁how', (7, 10)), ('▁are', (11, 14)), ('▁you?', (16, 20))]


SentencePiece  
- 모델과 함께 사용할 수 있는 텍스트 전처리를 위한 토큰화 알고리즘
- 텍스트를 일련의 유니코드 문자들로 간주하고 공백을 특수 문자인 _로 치환함
- Unigram 알고리즘과 함께 사용하면 사전 토큰화(pre-tokenization) 단계가 필요하지 않으므로 공백 문자가 사용되지 않는 언어(예: 중국어 또는 일본어)에 유용함
- 또 다른 주요 기능은 가역적 토큰화(reversible tokenization)임
- 공백에 대한 특별한 처리가 없기 때문에 토큰 디코딩은 토큰을 연결하고 _s를 공백으로 바꾸는 것으로 간단히 수행되고 정규화된 텍스트가 도출됨
- BERT 토크나이저는 반복되는 공백을 제거하므로 토큰화는 되돌릴 수 없음

토큰화 알고리즘  
- 학습 과정  
  - BPE : 소규모 vocabulary에서 시작하여 토큰 병합 규칙을 배움
  - WordPiece : 소규모 vocabulary에서 시작하여 토큰 병합 규칙을 배움
  - Unigram : 대규모 vocabulary에서 시작하여 토큰을 제거하는 규칙을 배움
- 학습 단계
  - BPE : 가장 빈번하게 발생되는 토큰 쌍을 병합함
  - WordPiece : 토큰 쌍의 빈도수에 기반한 최고 점수를 가진 쌍에 해당하는 토큰을 병합하고 각 개별 토큰의 빈도가 낮은 쌍에 특권을 부여함
  - Unigram : 전체 코퍼스에서 계산된 손실(loss)을 최소화하는 vocabulary의 모든 토큰을 제거함
- 학습 결과
  - BPE : 토큰 병합 규칙 및 vocabulary
  - WordPiece : Vocabulary
  - Unigram : 각 토큰에 대한 점수가 있는 vocabulary
- 인코딩
  - BPE : 단어를 문자들로 분할하고 학습 과정에서 습득한 병합 규칙 적용
  - WordPiece : Vocabulary의 시작 부분에서 시작하여 가장 긴 하위 단어(longest subword)를 찾은 다음 나머지 단어에 대해 동일한 작업을 수행
  - Unigram : 학습 과정에서 획득한 점수를 사용하여 가장 가능성 있는 토큰 분할을 찾음

Byte-Pair Encoding (BPE) 토큰화  

- 초기에 텍스트를 압축하는 알고리즘으로 개발된 후 GPT 모델을 사전 학습할 때 토큰화를 위해 OpenAI에서 사용되었음. 많은 트랜스포머 모델에서 사용됨
학습 알고리즘  
- BPE 학습은 정규화 및 사전 토큰화 단계가 완료된 후 말뭉치에 사용된 고유한 단어 집합을 계산하는 것으로 시작됨
- 그다음 이러한 단어들을 구성하는데 사용된 모든 기호(글자)를 바탕으로 vocabulary를 구축함  

"hug", "pug", "pun", "bun", "hugs"  
- 기본 vocabulary는 ["b", "g", "h", "n", "p", "s", "u"]가 됨
- 실제로는 기본 vocabulary에는 최소한 모든 ASCII 문자와 일부 유니코드 문자가 포함될 것임
- 토큰화하는 대상이 학습 말뭉치에 없는 문자를 사용하는 경우 해당 문자는 "알 수 없는 토큰(unknown token)"으로 변환됨
  - GPT-2 및 RoBERTa 토크나이저는 단어를 유니코드 문자가 아닌 바이트 단위로 구성된 것으로 간주함
  - 이 방법으로 기본 vocabulary는 작은 크기(256)를 갖지만 생각할 수 있는 모든 문자들이 여전히 포함될 수 있으며 알 수 없는 토큰으로 변환되지 않음
  - 이 트릭(trick)을 byte-level BPE 라고 함
- 이 기본 vocabulary를 구한 후, 기존 vocabulary의 두 요소를 새로운 것으로 병합하는 규칙인 merges 를 학습함으로써 원하는 vocabulary 크기에 도달할 때까지 새 토큰을 추가함
- 따라서 처음에는 이러한 병합으로 두 개의 문자가 있는 토큰이 생성되고 학습이 진행됨에 따라 더 긴 하위 단어(subwords)가 생성됨
- 토크나이저 학습 과정에서 어떤 단계에서든 BPE 알고리즘은 가장 빈번하게 출현하는 토큰 쌍을 검색함
  - "쌍"은 한 단어에서 두 개의 연속 토큰을 의미
  - 토큰은 처음에는 단일 문자임. 검색된 고빈도 토큰 쌍이 병합되며 이러한 과정이 계속 반복됨
- 이전 예제로 돌아가서 각 단어들의 출현빈도가 다음과 같다고 가정  

("hug", 10), ("pug", 5), ("pun", 12), ("bun", 4), ("hugs", 5)  

- 말뭉치 내에 "hug"가 10번, "pug"가 5번, "pun"이 12번, "bun"이 4번, "hugs"가 5번 출현한다는 의미
- 각 단어를 토큰의 목록으로 볼 수 있도록 각 단어를 문자로 분할하여 학습을 시작함  

("h" "u" "g", 10), ("p" "u" "g", 5), ("p" "u" "n", 12), ("b" "u" "n", 4), ("h" "u" "g" "s", 5)  
  
- ("h", "u")은 "hug" 및 "hugs"라는 단어에 존재하므로 말뭉치에서 총 15번 출현했음
  - 가장 빈번한 쌍은 아님
- 가장 빈번하게 출현하는 쌍은 "hug", "pug" 및 "hugs"에 있는 ("u", "g")이며 총 20번 출현함
- 따라서 토크나이저가 학습한 첫 번째 병합 규칙은 ("u", "g") -> "ug"이며, 이는 "ug"가 vocabulary에 추가되고 코퍼스 내의 모든 단어에서 "u"와 "g"가 병합되어야 함을 의미함
- 이 단계가 끝나면 vocabulary와 말뭉치가 다음과 같이 변경됨  

Vocabulary: ["b", "g", "h", "n", "p", "s", "u", "ug"]  
Corpus: ("h" "ug", 10), ("p" "ug", 5), ("p" "u" "n", 12), ("b" "u" "n", 4), ("h" "ug" "s", 5)  
  
- 이제 2개의 문자보다 더 긴 토큰이 생성되는 몇 가지 쌍이 존재함
- 예를 들어, ("h", "ug")로서 말뭉치에 15번 출현함
- 이 단계에서 가장 빈번하게 출현된 쌍은 ("u", "n")로서 말뭉치에 16번 나타나므로 학습된 두 번째 병합 규칙은 ("u", "n") -> "un"
- 이를 vocabulary에 추가하고 기존의 모든 항목을 병합하면 다음과 같이 됨  
  
Vocabulary: ["b", "g", "h", "n", "p", "s", "u", "ug", "un"]  
Corpus: ("h" "ug", 10), ("p" "ug", 5), ("p" "un", 12), ("b" "un", 4), ("h" "ug" "s", 5)  
  
- 이제 가장 빈번한 쌍은 ("h", "ug")이므로 병합 규칙("h", "ug") -> "hug"을 학습함
- 3글자로 구성된 토큰이 만들어지고 병합 후 코퍼스는 다음과 같음  
  
Vocabulary: ["b", "g", "h", "n", "p", "s", "u", "ug", "un", "hug"]  
Corpus: ("hug", 10), ("p" "ug", 5), ("p" "un", 12), ("b" "un", 4), ("hug" "s", 5)  
  
- 원하는 vocabulary 크기에 도달할 때까지 이 작업을 계속함

토큰화 알고리즘  
- 토큰화는 다음 단계를 적용하여 새로운 입력을 토큰화한다는 점에서 앞에서 살펴본 학습 프로세스와 밀접하게 연관되어 있음
  1. 정규화 (Normalization)
  2. 사전 토큰화 (Pre-tokenization)
  3. 단어를 개별 문자들로 분할
  4. 해당 분할에 순서대로 학습된 병합 규칙 적용
- 위에서 학습된 3가지 병합 규칙을 적용  

("u", "g") -> "ug"  
("u", "n") -> "un"  
("h", "ug") -> "hug"  
  
- "bug"라는 단어는 ["b", "ug"]로 토큰화됨
- "mug"는 기본 vocabulary에 문자 "m"이 없었기 때문에 ["[UNK]", "ug"]로 토큰화됨
- "thug"라는 단어는 ["[UNK]", "hug"]로 토큰화됨
- 문자 "t"는 기본 vocabulary에 없으며 병합 규칙을 적용하면 먼저 "u"와 "g"가 병합된 다음 "hu"와 "g"가 병합됨

In [19]:
# BPE 구현
corpus = [
    "This is the Hugging Face course.",
    "This chapter is about tokenization.",
    "This section shows several tokenizer algorithms.",
    "Hopefully, you will be able to understand how they are trained and generate tokens.",]

# pre-tokenize. gpt2 tokenizer 사용
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('gpt2')

# 말뭉치에 있는 각 단어의 빈도를 계산
from collections import defaultdict
word_freqs = defaultdict(int)

for text in corpus:
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in words_with_offsets]
    for word in new_words:
        word_freqs[word] += 1

print(word_freqs)

defaultdict(<class 'int'>, {'This': 3, 'Ġis': 2, 'Ġthe': 1, 'ĠHugging': 1, 'ĠFace': 1, 'Ġcourse': 1, '.': 4, 'Ġchapter': 1, 'Ġabout': 1, 'Ġtokenization': 1, 'Ġsection': 1, 'Ġshows': 1, 'Ġseveral': 1, 'Ġtokenizer': 1, 'Ġalgorithms': 1, 'Hopefully': 1, ',': 1, 'Ġyou': 1, 'Ġwill': 1, 'Ġbe': 1, 'Ġable': 1, 'Ġto': 1, 'Ġunderstand': 1, 'Ġhow': 1, 'Ġthey': 1, 'Ġare': 1, 'Ġtrained': 1, 'Ġand': 1, 'Ġgenerate': 1, 'Ġtokens': 1})


In [20]:
# 말뭉치에 사용된 모든 문자로 구성된 기본 vocabulary를 구함
alphabet = []

for word in word_freqs.keys():
    for letter in word:
        if letter not in alphabet:
            alphabet.append(letter)
alphabet.sort()
print(alphabet)

# 모델이 사용하는 특수 토큰 추가. gpt-2는 "<|endoftext|>"만 사용
vocab = ["<|endoftext|>"] + alphabet.copy()

# 각 단어를 개별문자로 분할
splits = {word: [c for c in word] for word in word_freqs.keys()}

# 각 쌍의 빈도를 계산하는 함수
def compute_pair_freqs(splits):
    pair_freqs = defaultdict(int)
    for word, freq in word_freqs.items():
        split = splits[word]
        if len(split) == 1:
            continue
        for i in range(len(split) - 1):
            pair = (split[i], split[i+1])
            pair_freqs[pair] += freq
    return pair_freqs

# 초기 분할 후 일부 값 확인
pair_freqs = compute_pair_freqs(splits)
for i, key in enumerate(pair_freqs.keys()):
    print(f'{key}: {pair_freqs[key]}')
    if i > 3:
        break    

[',', '.', 'F', 'H', 'T', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y', 'z', 'Ġ']
('T', 'h'): 3
('h', 'i'): 3
('i', 's'): 5
('Ġ', 'i'): 2
('Ġ', 't'): 7


In [21]:
# 가장 빈번하게 출현하는 쌍 찾기
best_pair = ''
max_freq = None

for pair, freq in pair_freqs.items():
    if max_freq is None or max_freq < freq:
        best_pair = pair
        max_freq = freq

print(best_pair, max_freq)

# 학습할 첫번째 병합 추가
merges = {("Ġ", "t"): "Ġt"}
vocab.append('Ġt')

def merge_pair(a, b, splits):
    for word in word_freqs:
        split = splits[word]
        if len(split) == 1:
            continue
        
        i = 0
        while i < len(split) - 1:
            if split[i] == a and split[i + 1] == b:
                split = split[:i] + [a + b] + split[i + 2 :]
            else:
                i += 1
        splits[word] = split
    return splits

splits = merge_pair("Ġ", "t", splits)
print(splits['Ġtrained'])

('Ġ', 't') 7
['Ġt', 'r', 'a', 'i', 'n', 'e', 'd']


In [22]:
# 모든 병합을 학습할 때까지 반복하는 모듈 구성
vocab_size = 50
while len(vocab) < vocab_size:
    pair_freqs = compute_pair_freqs(splits)
    best_pair = ''
    max_freq = None
    for pair, freq in pair_freqs.items():
        if max_freq is None or max_freq < freq:
            best_pair = pair
            max_freq = freq
            
    splits = merge_pair(*best_pair, splits)
    merges[best_pair] = best_pair[0] + best_pair[1]
    vocab.append(best_pair[0] + best_pair[1])
    
print(merges)
print(vocab)

{('Ġ', 't'): 'Ġt', ('i', 's'): 'is', ('e', 'r'): 'er', ('Ġ', 'a'): 'Ġa', ('Ġt', 'o'): 'Ġto', ('e', 'n'): 'en', ('T', 'h'): 'Th', ('Th', 'is'): 'This', ('o', 'u'): 'ou', ('s', 'e'): 'se', ('Ġto', 'k'): 'Ġtok', ('Ġtok', 'en'): 'Ġtoken', ('n', 'd'): 'nd', ('Ġ', 'is'): 'Ġis', ('Ġt', 'h'): 'Ġth', ('Ġth', 'e'): 'Ġthe', ('i', 'n'): 'in', ('Ġ', 'c'): 'Ġc', ('Ġa', 'b'): 'Ġab', ('Ġtoken', 'i'): 'Ġtokeni'}
['<|endoftext|>', ',', '.', 'F', 'H', 'T', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y', 'z', 'Ġ', 'Ġt', 'is', 'er', 'Ġa', 'Ġto', 'en', 'Th', 'This', 'ou', 'se', 'Ġtok', 'Ġtoken', 'nd', 'Ġis', 'Ġth', 'Ġthe', 'in', 'Ġc', 'Ġab', 'Ġtokeni']


In [23]:
# 새로운 텍스트를 토큰화하기 위해서는 사전 토큰화(pre-tokenize)하고 
# 분할(split)한 다음 학습한 모든 병합 규칙(merge rules)을 적용하면 됨
def tokenize(text):
    pre_tokenize_result = tokenizer._tokenizer.pre_tokenizer.pre_tokenize_str(text)
    pre_tokenized_text = [word for word, offset in pre_tokenize_result]
    splits = [[l for l in word] for word in pre_tokenized_text]
    for pair, merge in merges.items():
        for idx, split in enumerate(splits):
            i = 0
            while i < len(split) - 1:
                if split[i] == pair[0] and split[i + 1] == pair[1]:
                    split = split[:i] + [merge] + split[i + 2 :]
                else:
                    i += 1
            splits[idx] = split

    return sum(splits, [])

tokenize("This is not a token.")

['This', 'Ġis', 'Ġ', 'n', 'o', 't', 'Ġa', 'Ġtoken', '.']

WordPiece 토큰화  
- Google이 BERT를 사전 학습하기 위해 개발한 토큰화 알고리즘. 그 이후로 BERT 기반의 많은 Transformer 모델에서 재사용

학습 알고리즘  
- BPE와 마찬가지로 WordPiece는 모델에서 사용하는 특수 토큰과 초기 알파벳을 포함한 작은 vocabulary에서 시작함 - 접두사(예: BERT의 ##)를 추가하여 하위 단어(subwords)를 식별하기 때문에 각 단어는 처음에 해당 접두사를 단어 내부의 모든 문자에 추가하여 분할됨
- 예를 들어 "word"는 다음과 같이 분할됨
  - w ##o ##r ##d
- 초기 알파벳에는 단어의 시작 부분에 있는 모든 문자들(예: 'w')과 WordPiece 접두사가 선행하는 단어 내부에 있는 문자(예: 'o', 'r', 'd')가 포함됨
- 그다음 병합 규칙을 학습함
- BPE와 차이점은 병합할 쌍이 선택되는 방식
  - 가장 빈번하게 출현하는 쌍을 선택하는 대신 다음 공식을 사용하여 각 쌍에 대한 점수를 계산함
  - $score = \frac{freq\,of\,pair}{freq\,of\,first\,element \times freq\,of\,second\,element}$
- 쌍의 빈도를 각 부분의 빈도의 곱으로 나눔으로써, 알고리즘은 각 개별 부분들의 빈도가 낮은 쌍의 병합에 높은 우선순위를 부여함
  - vocabulary 내에서의 출현 빈도가 높은 ("un", "##able") 쌍을 굳이 병합할 필요는 없는데 그 이유는 "un"과 "##able" 각각이 다른 단어 내에서 매우 빈번하게 출현하여 높은 빈도를 나타내기 때문
  - "hu"와 "##gging"은 각각 자주 사용되지 않기 때문에 ("hu", "##gging")과 같은 쌍은 아마 더 빨리 병합될 것
    - "hugging"이라는 단어가 어휘에 자주 등장한다고 가정
- BPE와 동일한 vocabulary를 사용한 예시  

("hug", 10), ("pug", 5), ("pun", 12), ("bun", 4), ("hugs", 5)  
  
- 분할 결과는 다음과 같음  
  
("h" "##u" "##g", 10), ("p" "##u" "##g", 5), ("p" "##u" "##n", 12), ("b" "##u" "##n", 4), ("h" "##u" "##g" "##s", 5)  
  
- 초기 vocabulary는 ["b", "h", "p", "##g", "##n", "##s", "##u"]가 됨.(특수 토큰은 일단 제외)
- 가장 빈번한 쌍은 ("##u", "##g", 현재 20회)이지만 "##u"의 개별 빈도가 매우 높아 점수가 가장 높지는 않음. 
  - ("##u", "##g") = 36, ("##u") = 36, ("##g") = 20
  - 36 / (36 * 20) = 0.05
- "##u"가 포함된 모든 쌍은 실제로 동일한 점수(36/36*36)를 가지므로 가장 좋은 점수는 ("##g", "##s")임(36/36*20).
  - 이것은 "##u"가 없는 유일한 쌍. 학습된 첫 번째 병합은 ("##g", "##s") -> ("##gs")임.
- 병합할 때 두 토큰 사이의 ##을 제거하므로 "##gs"를 추가하고 말뭉치의 모든 단어에 적용  
  
Vocabulary: ["b", "h", "p", "##g", "##n", "##s", "##u", "##gs"]  
Corpus: ("h" "##u" "##g", 10), ("p" "##u" "##g", 5), ("p" "##u" "##n", 12), ("b" "##u" "##n", 4), ("h" "##u" "##gs", 5)  
  
- 이 시점에서 "##u"는 가능한 모든 쌍에 있으므로 모두 동일한 점수를 가짐
- 이 경우 첫 번째 쌍이 병합되므로 ("h", "##u") -> "hu"가 학습됨. 그 결과는 다음과 같음  
  
Vocabulary: ["b", "h", "p", "##g", "##n", "##s", "##u", "##gs", "hu"]  
Corpus: ("hu" "##g", 10), ("p" "##u" "##g", 5), ("p" "##u" "##n", 12), ("b" "##u" "##n", 4), ("hu" "##gs", 5)  
  
- 이제 최고 점수는 ("hu", "##g") 및 ("hu", "##gs")가 동일하게 계산되므로(다른 모든 쌍의 경우 1/21이고 이 두 쌍은  1/15) 가장 큰 점수를 가진 첫 번째 쌍이 병합됨
  - ("hu", "##g") = 15 / 15 * 15, ("hu", "##gs") = 15 / 15 * 15
- 원하는 어휘 크기에 도달할 때까지 계속 반복  

Vocabulary: ["b", "h", "p", "##g", "##n", "##s", "##u", "##gs", "hu", "hug"]  
Corpus: ("hug", 10), ("p" "##u" "##g", 5), ("p" "##u" "##n", 12), ("b" "##u" "##n", 4), ("hu" "##gs", 5)  

토큰화 알고리즘  
- 토큰화는 WordPiece가 학습된 병합 규칙은 제외하고 최종 vocabulary만 저장한다는 점에서 BPE와는 다름
- 토큰화할 단어에서 시작하여 WordPiece는 vocabulary에 있는 가장 긴 하위 단어를 찾은 다음 분할함
  - 위의 예에서 학습한 vocabulary를 사용하는 경우, 단어 "hugs"의 경우 처음부터 시작하는 가장 긴 하위 단어는 vocabulary 내부에 있는 "hug"이므로 거기에서 분할하여 ["hug", "##s"]로 분할됨
  - 그 다음 "##s"가 vocabulary에 존재하고 이를 계속 사용할 수 있으므로 "hugs"의 토큰화 결과는 ["hug", "##s"]임.
- BPE를 사용하면 학습된 병합(merges)을 순서대로 적용하고 이를 ["hu", "##gs"]로 토큰화하므로 인코딩이 다르게 됨
- "bugs"라는 단어가 어떻게 토큰화되는지 확인해보면, "b"는 vocabulary에 존재하는 단어의 시작 부분에서 시작하는 가장 긴 하위 단어이므로 거기서 분할하여 ["b", "##ugs"]라는 중간 결과가 나옴
- 다음으로 "##u"는 vocabulary에 있는 "##ugs"의 시작 부분에서 시작하는 가장 긴 하위 단어이므로 거기에서 분할하여 ["b", "##u, "##gs"]를 얻음
- 마지막으로 "##gs"가 vocabulary에 있으므로 ["b", "##u, "##gs"]이 "bugs"의 토큰화 결과임
- 토큰화가 vocabulary에서 하위 단어(subword)를 더이상 찾을 수 없는 단계에 도달하면 전체 단어를 "unknown"으로 토큰화함
  - 예를 들어 "mug"는 "bum"과 마찬가지로 ["[UNK]"]로 토큰화됨
    - "b"와 "##u"로 시작할 수 있더라도 "##m"이 vocabulary에 존재하지 않으므로 결과 토큰화는 ["b", "##u", "[UNK]"]가 아니라 ["[UNK]"]입니다. 
  - 이것은 vocabulary에 없는 개별 문자만 "unknwon"으로 분류하는 BPE와의 다른 차이점

In [24]:
corpus = [
    "This is the Hugging Face course.",
    "This chapter is about tokenization.",
    "This section shows several tokenizer algorithms.",
    "Hopefully, you will be able to understand how they are trained and generate tokens.",]

# pre-tokenizer. bert와 같은 wordpiece 토크나이저를 구현하고 있음
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

# 각 단어의 빈도 계산
from collections import defaultdict
word_freqs = defaultdict(int)
for text in corpus:
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in words_with_offsets]
    for word in new_words:
        word_freqs[word] += 1
print(word_freqs)

defaultdict(<class 'int'>, {'This': 3, 'is': 2, 'the': 1, 'Hugging': 1, 'Face': 1, 'course': 1, '.': 4, 'chapter': 1, 'about': 1, 'tokenization': 1, 'section': 1, 'shows': 1, 'several': 1, 'tokenizer': 1, 'algorithms': 1, 'Hopefully': 1, ',': 1, 'you': 1, 'will': 1, 'be': 1, 'able': 1, 'to': 1, 'understand': 1, 'how': 1, 'they': 1, 'are': 1, 'trained': 1, 'and': 1, 'generate': 1, 'tokens': 1})


In [25]:
# 알파벳은 단어의 모든 첫 글자와 ## 접두사가 붙은 단어에 나타나는 다른 모든 글자로 구성된 고유한 집합
alphabet = []
for word in word_freqs.keys():
    if word[0] not in alphabet:
        alphabet.append(word[0])
    for letter in word[1:]:
        if f'##{letter}' not in alphabet:
            alphabet.append(f'##{letter}')
alphabet.sort()
print(alphabet)

# 특수 토큰 추가. BERT : ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"]
vocab = ['[PAD]', '[UNK],', '[CLS]', '[SEP]', '[MASK]'] + alphabet.copy()

# 접두사가 ##이 아닌 모든 문자를 사용하여 단어 분할
# 첫문자는 그대로, 나머지 문자는 ##를 붙여서.
splits = {word: [c if i == 0 else f'##{c}' for i, c in enumerate(word)] 
          for word in word_freqs.keys()}

# 각 쌍의 점수 계산 함수
def compute_pair_scores(splits):
    letter_freqs = defaultdict(int)
    pair_freqs = defaultdict(int)
    for word, freq in word_freqs.items():
        split = splits[word]
        if len(split) == 1:
            letter_freqs[split[0]] += freq
            continue
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            letter_freqs[split[i]] += freq
            pair_freqs[pair] += freq
        letter_freqs[split[-1]] += freq
    
    scores = {pair: freq / (letter_freqs[pair[0]] * letter_freqs[pair[1]])
              for pair, freq in pair_freqs.items()}
    return scores

pair_scores = compute_pair_scores(splits)
for i, key in enumerate(pair_scores.keys()):
    print(f'{key}: {pair_scores[key]}')
    if i >= 3:
        break

best_pair = ''
max_score = None
for pair, score in pair_scores.items():
    if max_score is None or max_score < score:
        best_pair = pair
        max_score = score

print(best_pair, max_score)

# 학습할 첫 번째 병합은 ('a', '##b') -> 'ab'이고 vocabulary에 'ab'를 추가
vocab.append('ab')

['##a', '##b', '##c', '##d', '##e', '##f', '##g', '##h', '##i', '##k', '##l', '##m', '##n', '##o', '##p', '##r', '##s', '##t', '##u', '##v', '##w', '##y', '##z', ',', '.', 'F', 'H', 'T', 'a', 'b', 'c', 'g', 'h', 'i', 's', 't', 'u', 'w', 'y']
('T', '##h'): 0.125
('##h', '##i'): 0.03409090909090909
('##i', '##s'): 0.02727272727272727
('i', '##s'): 0.1
('a', '##b') 0.2


In [26]:
# splits 딕셔너리에 병합 적용
def merge_pair(a, b, splits):
    for word in word_freqs:
        split = splits[word]
        if len(split) == 1:
            continue
        i = 0
        while i < len(split) - 1:
            if split[i] == a and split[i+1] == b:
                merge = a + b[2:] if b.startswith('##') else a + b
                split = split[:i] + [merge] + split[i+2:]
            else:
                i += 1
        splits[word] = split
    return splits

splits = merge_pair('a', '##b', splits)
print(splits['about'])

['ab', '##o', '##u', '##t']


In [27]:
vocab_size = 70
while len(vocab) < vocab_size:
    scores = compute_pair_scores(splits)
    best_pair, max_score = '', None
    for pair, score in scores.items():
        if max_score is None or max_score < score:
            best_pair = pair
            max_score = score
    splits = merge_pair(*best_pair, splits)
    new_token = (best_pair[0] + best_pair[1][2:]
                 if best_pair[1].startswith('##') 
                 else best_pair[0] + best_pair[1])
    vocab.append(new_token)
print(vocab)

['[PAD]', '[UNK],', '[CLS]', '[SEP]', '[MASK]', '##a', '##b', '##c', '##d', '##e', '##f', '##g', '##h', '##i', '##k', '##l', '##m', '##n', '##o', '##p', '##r', '##s', '##t', '##u', '##v', '##w', '##y', '##z', ',', '.', 'F', 'H', 'T', 'a', 'b', 'c', 'g', 'h', 'i', 's', 't', 'u', 'w', 'y', 'ab', '##fu', 'Fa', 'Fac', '##ct', '##ful', '##full', '##fully', 'Th', '##hm', '##thm', 'Hu', 'Hug', 'Hugg', 'ch', 'cha', 'chap', 'chapt', 'sh', 'th', 'is', '##thms', '##za', '##zat', '##ut', '##ta']


In [28]:
# 새로운 텍스트를 토큰화하기 위해 사전 토큰화하고(pre-tokenization), 분할한 다음(split),
# 각 단어에 토큰화 알고리즘을 적용. 즉, 첫번째 단어의 시작 부분에서 시작하는 가장 큰 하위 
# 단어를 찾아 분할한 다음, 두번째 부분에서 프로세스를 반복하고 나머지 단어와 텍스트의 
# 다음 단어에 대해 계속 반복함
def encode_word(word):
    tokens = []
    while len(word) > 0:
        i = len(word)
        while i > 0 and word[:i] not in vocab:
            i -= 1
        if i == 0:
            return ['[UNK]']
        tokens.append(word[:i])
        word = word[i:]
        if len(word) > 0:
            word = f'##{word}'
    return tokens

# vocabulary에 존재하는 단어와 아닌 단어 테스트
print(encode_word('Hugging'))
print(encode_word('HOgging'))

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

# 텍스트 토큰화
def tokenize(text):
    pre_tokenize_result = tokenizer._tokenizer.pre_tokenizer.pre_tokenize_str(text)
    pre_tokenized_text = [word for word, offset in pre_tokenize_result]
    encoded_words = [encode_word(word) for word in pre_tokenized_text]
    return sum(encoded_words, [])

# 테스트
print(tokenize('This is the Hugging Face course!'))

['Hugg', '##i', '##n', '##g']
['[UNK]']
['Th', '##i', '##s', 'is', 'th', '##e', 'Hugg', '##i', '##n', '##g', 'Fac', '##e', 'c', '##o', '##u', '##r', '##s', '##e', '[UNK]']


Unigram 토큰화  
- Unigram 알고리즘은 AlBERT, T5, mBART, Big Bird 및 XLNet과 같은 모델에서 사용되는 토큰화 알고리즘인 SentencePiece에서 자주 사용됨
학습 알고리즘  
- BPE 및 WordPiece와 비교하여 Unigram은 다른 방향으로 작동함
- 크기가 큰 vocabulary에서 시작하여 원하는 vocabulary 크기에 도달할 때까지 토큰을 제거함
- 기본 vocabulary를 구축하는데 사용할 수 있는 몇 가지 옵션이 있음
  - 사전 토큰화된 단어에서 가장 일반적인 부분 문자열을 취하거나 큰 규모의 vocabulary를 가진 초기 말뭉치에 BPE를 적용할 수 있음
- 학습의 각 단계에서 Unigram 알고리즘은 현재 vocabulary가 주어졌을 때의 말뭉치에 대한 손실(loss)을 계산함
- 그 다음 vocabulary의 각 기호(symbol)에 대해 해당 기호가 제거되면 전체 손실이 얼마나 증가할지 계산하고 가장 적게 증가하는 기호를 찾음
- 이렇게 찾은 기호들(symbols)은 말뭉치에 대한 전체 손실에 더 적은 영향을 미치므로 어떤 의미에서는 "덜 필요(less needed)"하고 제거 대상으로 가장 적합한 후보임
- 이것은 비용이 많이 드는 작업이므로 가장 낮은 손실을 초래하는 기호 하나만 제거하지 않고, 이러한 기호들의 
p%(p는 제어할 수 있는 하이퍼파라미터이고 일반적으로 10 또는 20을 지정)를 제거함
- 이 과정은 vocabulary가 원하는 크기에 도달할 때까지 반복됨
- 모든 단어를 토큰화할 수 있도록 기본 문자들을 제거하지 않는다는 점도 중요함
- 이전 예제의 말뭉치를 재사용  

("hug", 10), ("pug", 5), ("pun", 12), ("bun", 4), ("hugs", 5)  
  
- 초기 vocabulary는 위 말뭉치에 존재하는 모든 단어들의 모든 하위 문자열(substrings)로 구성됨
  
["h", "u", "g", "hu", "ug", "p", "pu", "n", "un", "b", "bu", "s", "hug", "gs", "ugs"]  
  
토큰화 알고리즘  
- Unigram 모델은 개별 토큰들의 출현 분포가 서로 독립적이고 같은 확률분포를 가진다는 가정을 하는 언어 모델 유형
- 토큰 X의 확률이 문맥에 상관없이 동일함
- Unigram 언어 모델을 사용하여 텍스트를 생성하는 경우 항상 가장 일반적이고 흔한(common) 토큰을 도출함
- 특정 토큰의 확률은 말뭉치 내에서의 해당 토큰 출현 빈도를 vocabulary에 존재하는 모든 토큰들의 출현 빈도의 합으로 나눈 것(확률의 합이 1이 되도록 하기 위해)임
- 예를 들어 "ug"는 "hug", "pug" 및 "hugs"에 있으므로 말뭉치에서의 빈도는 20
- 다음은 vocabulary에서의 모든 하위 단어(subwords)의 빈도  
  
("h", 15) ("u", 36) ("g", 20) ("hu", 15) ("ug", 20) ("p", 17) ("pu", 17) ("n", 16) ("un", 16) ("b", 4) ("bu", 4) ("s", 5) ("hug", 15) ("gs", 5) ("ugs", 5)  
  
- 모든 빈도의 합은 210이고 하위 단어(subword) "ug"의 확률은 20/210
- 이제 주어진 단어를 토큰화하기 위해, 가능한 모든 토큰 분할을 Unigram 모델에 따라 각각의 확률을 계산 
- 모든 토큰의 출현 빈도가 독립적인 것으로 간주되기 때문에 이 확률은 각 토큰의 확률의 곱임
  - 예를 들어, "pug"의 토큰화 결과인 ["p", "u", "g"]의 확률은 다음과 같이 계산됨   
    - P(["p", "u", "g"]) = P("p") * P("u") * P("g") = $\frac{17}{210} \times \frac{36}{210} \times \frac{20}{210} = 0.001321 $
  - 토큰화 결과인 ["pu", "g"]의 확률은 다음과 같음
    - P(["pu", "g"]) = P("pu") * P("g") = $\frac{17}{210} \times \frac{20}{210} = 0.007709 $ 
- ["pu", "g"]이 훨씬 더 자주 출현한다고 볼 수 있음
  - 일반적으로 가장 적은 수의 하위 토큰들로 구성된 토큰화 결과는 비교적 높은 확률을 가짐
- Unigram 모델을 사용한 단어의 토큰화는 가장 높은 확률을 나타내는 분할 형태로 토큰화됨
- "pug"의 예에서 가능한 각 분할에 대해 얻을 수 있는 확률은 다음과 같음
  - ["p", "u", "g"] : 0.000389
  - ["p", "ug"] : 0.0022676
  - ["pu", "g"] : 0.0022676
- "pug"는 위 분할 방법 중에서 ["p", "ug"] 또는 ["pu", "g"]로 토큰화될 수 있음. 그러나 큰 규모의 말뭉치에서는 분할 방법에 대한 확률값이 같은 경우가 매우 드뭄
- 위의 경우에서는 가능한 모든 분할을 찾고 확률을 계산하는 것이 쉬웠지만, 일반적으로는 더 어려울 수 있음
- 이를 위해 사용되는 Viterbi 알고리즘이 있음
  - 본질적으로 주어진 단어에 대한 가능한 모든 분할들을 나타낼 수 있는 그래프를 구성할 수 있음
  - 만일 주어진 단어 내의 문자 a에서 b까지의 하위 단어(subword)가 vocabulary에 존재한다면 우리는 이 그래프 내에서 a에서 출발하여 b까지 가는 그래프 내의 가지(branch)가 있다고 말할 수 있음
  - 그리고 이 하위 단어의 확률을 해당 가지(branch)에 지정할 수 있음
- 그래프에서 최고 점수를 얻을 경로를 찾기 위해 Viterbi 알고리즘은 단어 내의 각 위치(문자)에 대해 해당 위치에서 끝나는 경로의 최고 점수를 나타내는 분할(segmentation)을 결정함
- 단어의 처음 위치부터 끝까지 이동하면서, 현재 위치에서 끝나는 모든 하위 단어를 검사한 다음 이 하위 단어가 시작하는 위치에서 최고의 토큰화 점수를 사용하여 최상의 점수를 찾을 수 있음
- 그런 다음 끝에 도달하기 위해 선택한 경로를 펼치면 됨
- 앞에서 구성한 vocabulary와 "unhug"라는 단어를 사용한 예를 보고 해당 단어의 각 위치에 대해 최고 점수로 끝나는 하위 단어는 다음과 같음
  - Character 0 (u): "u" (score 0.171429)
  - Character 1 (n): "un" (score 0.076191)
  - Character 2 (h): "un" "h" (score 0.005442)
  - Character 3 (u): "un" "hu" (score 0.005442)
  - Character 4 (g): "un" "hug" (score 0.005442)
- 위에서 처음부터 마지막 글자('g')까지 진행한 결과, ["un", "hug"]가 가장 높은 점수인 0.005442를 나타냄. "unhug"는 ["un", "hug"]로 토큰화됨
다시 학습 알고리즘  
- 학습 과정에서 사용된 손실(loss)값을 확인
- 각각의 주어진 단계에서 이 손실은 말뭉치 내의 모든 단어를 토큰화하여 계산됨
- 계산 과정에서 앞에서 설명한 것처럼 현재 vocabulary와 말뭉치에 있는 각 토큰의 빈도에 의해 결정된 유니그램 모델을 사용함
- 말뭉치의 각 단어별로 점수를 계산하며 손실은 해당 점수의 음의 로그 우도(negative log likelihood)
  - 말뭉치에 있는 모든 단어의 -log(P(word)) 합계임  

("hug", 10), ("pug", 5), ("pun", 12), ("bun", 4), ("hugs", 5)  
- 각 단어의 토큰화 결과 및 점수는 다음과 같음
  - "hug": ["hug"] (score 0.071428)
  - "pug": ["pu", "g"] (score 0.007710)
  - "pun": ["pu", "n"] (score 0.006168)
  - "bun": ["bu", "n"] (score 0.001451)
  - "hugs": ["hug", "s"] (score 0.001701)
- 손실(loss)은 다음과 같음  
  
10 * (-log(0.071428)) + 5 * (-log(0.007710)) + 12 * (-log(0.006168)) + 4 * (-log(0.001451)) + 5 * (-log(0.001701)) = 169.8  
  
- 이제 각 토큰을 제거하는 것이 손실값에 어떤 영향을 미치는지 계산해야 함
- 이 작업은 수작업으로 하기에는 시간이 걸리므로 여기에서는 두 개의 토큰("pu", "hug")에 대해 이를 수행하고 나머지 프로세스는 아래에서 해당 작업에 대한 실제 구현이 완료되었을 때 수행함
- 이 시점에서 "pug"는 동일한 점수(0.0022676)를 가진 두개의 토큰화 결과를 가지고 있음. ["p", "ug"]와 ["pu", "g"] 값임
- 따라서 "pu" 토큰을 vocabulary에서 제거하더라도 토큰화 결과가 동일한 점수를 가진 ["p", "ug"]가 되므로 결론적으로는 위에서 계산한 것과 똑같은 손실(loss)값이 도출됨
- 반면에, "hug"를 제거하면 손실값이 더 올라가게 됨. 그 이유는 "hug"와 "hugs"의 토큰화 결과가 다음과 같이 되기 때문임
  - "hug": ["hu", "g"] (score 0.006802)   # 위의 토큰화 결과보다 점수가 낮아짐.
  - "hugs": ["hu", "gs"] (score 0.001701)
- 그 결과 다음 계산 결과값 만큼 손실값이 올라가게 됨
  - 10 * (-log(0.071428)) + 10 * (-log(0.006802)) = 23.5
- 결론적으로, 토큰 "pu"는 vocabulary에서 제거되겠지만 "hug"는 제거되지 않음

In [29]:
corpus = [
    "This is the Hugging Face course.",
    "This chapter is about tokenization.",
    "This section shows several tokenizer algorithms.",
    "Hopefully, you will be able to understand how they are trained and generate tokens.",]

# xlnet-base-cased 모델 사용
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('xlnet-base-cased')

# 출현 빈도 계산
from collections import defaultdict
word_freqs = defaultdict(int)
for text in corpus:
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in words_with_offsets]
    for word in new_words:
        word_freqs[word] += 1
print(word_freqs)

defaultdict(<class 'int'>, {'▁This': 3, '▁is': 2, '▁the': 1, '▁Hugging': 1, '▁Face': 1, '▁course.': 1, '▁chapter': 1, '▁about': 1, '▁tokenization.': 1, '▁section': 1, '▁shows': 1, '▁several': 1, '▁tokenizer': 1, '▁algorithms.': 1, '▁Hopefully,': 1, '▁you': 1, '▁will': 1, '▁be': 1, '▁able': 1, '▁to': 1, '▁understand': 1, '▁how': 1, '▁they': 1, '▁are': 1, '▁trained': 1, '▁and': 1, '▁generate': 1, '▁tokens.': 1})


In [30]:
# 원하는 크기보다 더 크게 vocabulary를 초기화해야 하고 모든 기본 문자들을 포함해야 함
# 그렇지 않으면 모든 단어를 토큰화할 수 없음
# 또한 길이가 더 긴 부분 문자열에 대해서는 가장 빈번하게 출현하는 것들만 추가할 것이므로 
# 먼저 빈도순으로 정렬함
char_freqs = defaultdict(int)
subwords_freqs = defaultdict(int)
for word, freq in word_freqs.items():
    for i in range(len(word)):
        char_freqs[word[i]] += freq
        # 길이가 적어도 2 이상인 subword들을 추가함
        for j in range(i+2, len(word)+1):
            subwords_freqs[word[i:j]] += freq
# subword들을 빈도 역순으로 정렬
sorted_subwords = sorted(
    subwords_freqs.items(), key=lambda x: x[1], reverse=True)
print(sorted_subwords[:5])

# 크기가 300인 초기 vocabulary를 구성하기 위해서 앞에서 만들어진 sorted_subwords 중
# 빈도가 높은 하위 단어들을 선택함
token_freqs = list(char_freqs.items()) + sorted_subwords[: 300 - len(char_freqs)]
token_freqs = {token: freq for token, freq in token_freqs}

# 모든 빈도의 합을 계산하여 빈도를 확률로 변환함. 확률의 로그값 저장
# 작은 숫자를 곱하는 것보다 로그를 더하는 것이 수치적으로 안정적이기 때문
# 이렇게 하면 모델 손실 계산이 단순화됨
from math import log
total_sum = sum([freq for token, freq in token_freqs.items()])
model = {token: -log(freq / total_sum) for token, freq in token_freqs.items()}

[('▁t', 7), ('is', 5), ('er', 5), ('▁a', 5), ('▁to', 4)]


- 다음으로 Viterbi 알고리즘을 사용하여 단어를 토큰화하는 기능
  - 이 알고리즘은 단어의 각 부분 문자열에 대한 최상의 분할을 계산하고 이를 best_segmentations라는 변수에 저장함
  - 단어의 각 위치당 하나의 딕셔너리(0에서 전체 길이까지)을 두 개의 키와 함께 저장함
    - 키 값은 가장 점수가 높은 분할(segmentation)에서 마지막 토큰의 시작 인덱스와 해당 점수임
  - 마지막 토큰의 시작 인덱스를 사용하여 목록이 완전히 채워지면 전체 분할을 검색할 수 있음
- 목록 채우기는 단 2개의 루프로 완료됨
- 기본 루프는 각 시작 위치로 이동하고 두 번째 루프는 해당 시작 위치에서 시작하는 모든 하위 문자열을 검토함
- 하위 문자열이 vocabulary에 있는 경우 해당 끝 위치까지 단어의 새로운 분할이 있으며 이를 best_segmentations에 있는 것과 비교함

In [31]:
# 메인 루프가 끝나면 단어의 시작 부분에 도달할 때까지 끝에서 시작하여 특정 시작 위치에서 다음 위치로 이동하면서 토큰을 기록합니다:
def encode_word(word, model):
    best_segmentations = [{'start': 0, 'score': 1}] + \
        [{'start': None, 'score': None} for _ in range(len(word))]
    for start_idx in range(len(word)):
        # this should be properly filled by the previous steps of the loop
        best_score_at_start = best_segmentations[start_idx]['score']
        for end_idx in range(start_idx+1, len(word)+1):
            token = word[start_idx:end_idx]
            if token in model and best_score_at_start is not None:
                score = model[token] + best_score_at_start
                # if we have found a better segmentation ending at end_idx, we update
                if (best_segmentations[end_idx]['score'] is None or
                    best_segmentations[end_idx]['score'] > score):
                    best_segmentations[end_idx] = {
                        'start': start_idx, 'score': score}
    segmentation = best_segmentations[-1]
    if segmentation['score'] is None:
        # we didn`t find a tokenization of the word -> unknown
        return ['<unk>'], None

    score = segmentation['score']
    start = segmentation['start']
    end = len(word)
    tokens = []
    while start != 0:
        tokens.insert(0, word[start:end])
        next_start = best_segmentations[start]['start']
        end = start
        start = next_start
    tokens.insert(0, word[start:end])
    return tokens, score

# 테스트
print(encode_word('Hopefully', model))
print(encode_word('This', model))

(['H', 'o', 'p', 'e', 'f', 'u', 'll', 'y'], 41.54264024176184)
(['This'], 6.29162837839724)


In [32]:
# 손실 계산
def compute_loss(model):
    loss = 0
    for word, freq in word_freqs.items():
        _, word_loss = encode_word(word, model)
        loss += freq * word_loss
    return loss

# 테스트
compute_loss(model)

413.362600202517

In [33]:
# 각 토큰 점수 계산
import copy
def compute_scores(model):
    scores = {}
    model_loss = compute_loss(model)
    for token, score in model.items():
        # we always keep tokens of length 1
        if len(token) == 1:
            continue
        model_without_token = copy.deepcopy(model)
        _ = model_without_token.pop(token)
        scores[token] = compute_loss(model_without_token) - model_loss
    return scores

scores = compute_scores(model)
print(scores['ll'])
print(scores['his'])

6.383135099029346
0.0


In [34]:
# 모델에서 특수 토큰 추가하고 원하는 크기에 도달할 때까지 
# vocabulary에서 토큰을 반복적으로 제거해 나감
percent_to_remove = 0.1
while len(model) > 100:
    scores = compute_scores(model)
    sorted_scores = sorted(scores.items(), key=lambda x: x[1])
    # Remove percent_to_remove tokens with the lowest scores.
    for i in range(int(len(model) * percent_to_remove)):
        _ = token_freqs.pop(sorted_scores[i][0])

    total_sum = sum([freq for token, freq in token_freqs.items()])
    model = {token: -log(freq / total_sum) for token, freq in token_freqs.items()}

def tokenize(text, model):
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    pre_tokenized_text = [word for word, offset in words_with_offsets]
    # 입력 텍스트에 사전 토큰화를 적용한 다음 토큰화 진행
    encoded_words = [encode_word(word, model)[0] for word in pre_tokenized_text]
    return sum(encoded_words, [])

tokenize("This is the Hugging Face course.", model)

['▁This', '▁is', '▁the', '▁Hugging', '▁Face', '▁course.']

블록 단위로 토크나이저 빌딩하기  
- Tokenizer 라이브러리는 Tokenizer 클래스를 중심으로 다양한 하부 모듈들이 기능별로 결합된 구성 요소(building blocks)가 결합된 형태로 구축되어 있음
  - 정규화 : 공백이나 악센트 제거, 유니코드 정규화 등과 같은 모든 텍스트 정제 작업
  - 사전 토큰화 : 입력을 단어들로 분리
  - 모델을 통한 입력 실행 : 사전 토큰화된 단어들을 사용하여 토큰 시퀀스 생성
    - BPE, WordPiece 및 Unigram과 같이 사용할 수 있는 다양한 유형의 Model
    - trainer에는 말뭉치에서 모델을 훈련하는데 사용할 수 있는 Trainer가 포함
  - 후처리 : 토큰나이저의 특수 토큰 추가, attention mask 및 토큰 유형 ID 생성

In [35]:
# 말뭉치 확보
from datasets import load_dataset
dataset = load_dataset('wikitext', name='wikitext-2-raw-v1', split='train')

# 1000개의 텍스트 배치를 생성하는 generator
def get_training_corpus():
    for i in range(0, len(dataset), 1000):
        yield dataset[i : i + 1000]['text']
        
# 텍스트 파일에서 직접 학습할 수도 있음.
with open('wikitext-2.txt', 'w', encoding='utf-8') as f:
    for i in range(len(dataset)):
        f.write(dataset[i]['text'] + '\n')

Found cached dataset wikitext (/root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)


In [36]:
# wordpiece 토크나이저 빌딩
from tokenizers import decoders, models, normalizers, pre_tokenizers
from tokenizers import processors, trainers, Tokenizer

tokenizer = Tokenizer(models.WordPiece(unk_token='[UNK]'))

# 모델이 이전에 본 적이 없는 문자를 만났을 때 무엇을 반환할지 알 수 있도록 unk_token을 
# 지정해야 함. 여기서 설정할 수 있는 다른 인수에는 모델의 vocab과 각 단어의 최대 
# 길이를 지정하는 max_input_chars_per_word(이 길이보다 더 긴 단어는 분할됨)가 포함됨.

# 토큰화의 첫 번째 단계는 정규화(normalization)
# BERT에 대해 설정할 수 있는 기본적인 옵션이 포함된 BertNormalizer가 존재함.
# 지정할 수 있는 옵션에는 lowercase, strip_accents 등을 비롯하여, 모든 제어 문자를 
# 제거하고 반복되는 공백을 단일 문자로 바꾸는 clean_text 그리고 
# 한자(Chinese characters) 주위에 공백을 배치하는 handle_chinese_chars가 있음.
# bert-base-uncased 토크나이저를 복제하려면 이 노멀라이저(normalizer)를 설정하면 됨.
tokenizer.normalizer = normalizers.BertNormalizer(lowercase=True)

# 일반적으로 새로운 토크나이저를 빌드할 때 Tokenizers 라이브러리에 이미 구현된 
# 편리한 노멀라이저에 접근할 수 없음. 따라서 BERT 노멀라이저를 직접 만들어야 함.
# NFD 유니코드 노멀라이저를 포함하지 않으면 악센트가 있는 문자를 인식하지 못함.
tokenizer.normalizer = normalizers.Sequence(
    [normalizers.NFD(), normalizers.Lowercase(), normalizers.StripAccents()])

In [37]:
# 사전 토큰화 단계.
tokenizer.pre_tokenizer = pre_tokenizers.BertPreTokenizer()

# 또는 처음부터 빌드할 수 있음.
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

# 공백, 문자, 숫자, 밑줄문자가 아닌 모든 문자를 기준으로 텍스트를 분할하므로,
# 결국은 공백과 구두점으로 분할한다고 볼 수 있음
print(tokenizer.pre_tokenizer.pre_tokenize_str("Let's test my pre-tokenizer."))

# Sequence를 사용하여 여러 사전 토크나이저들을 결합할 수도 있음
pre_tokenizer = pre_tokenizers.Sequence(
    [pre_tokenizers.WhitespaceSplit(), pre_tokenizers.Punctuation()])
print(pre_tokenizer.pre_tokenize_str("Let's test my pre-tokenizer."))

[('Let', (0, 3)), ("'", (3, 4)), ('s', (4, 5)), ('test', (6, 10)), ('my', (11, 13)), ('pre', (14, 17)), ('-', (17, 18)), ('tokenizer', (18, 27)), ('.', (27, 28))]
[('Let', (0, 3)), ("'", (3, 4)), ('s', (4, 5)), ('test', (6, 10)), ('my', (11, 13)), ('pre', (14, 17)), ('-', (17, 18)), ('tokenizer', (18, 27)), ('.', (27, 28))]


In [38]:
# 토큰화 파이프라인 중 모델을 통해 입력을 실행하는 단계.
# 이를 위해서 WordPieceTrainer가 필요함. Tokenizers에서 트레이너(trainer)를 
# 인스턴스화할 때는 사용하려는 모든 특수 토큰을 전달해야 함
# 그렇지 않으면 이들 토큰이 학습 코퍼스에 없으므로 vocabulary에 추가되지 않음
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.WordPieceTrainer(
    vocab_size=25000, special_tokens=special_tokens)

# vocab_size 및 special_tokens를 지정하는 것 외에도 min_frequency(토큰이 vocabulary에
# 포함되기 위한 최소 출현 빈도)를 설정하거나 continue_subword_prefix(##말고 다른 기호를 
# 사용하려는 경우)를 변경할 수 있음

# iterator를 사용하여 모델 학습
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

# 텍스트 파일을 사용하여 토크나이저를 학습할 수 있음
tokenizer.model = models.WordPiece(unk_token='[UNK]')
tokenizer.train(['wikitext-2.txt'], trainer=trainer)

# encode()를 호출하여 텍스트에서 토크나이저를 테스트할 수 있음
# encoding은 다양한 속성(ids, type_ids, tokens, offsets, attention_mask, 
# special_tokens_mask, overflowing)에 토크나이저의 필요한 출력을 포함하고 있는 
# Encoding 클래스 객체.
encoding = tokenizer.encode("Let's test this tokenizer.")
print(encoding.tokens)







['let', "'", 's', 'test', 'this', 'tok', '##eni', '##zer', '.']


In [39]:
# 토큰화 파이프라인의 후처리(post-processing) 단계.
# 시작 부분에 [CLS] 토큰을 추가하고 끝 부분에 [SEP] 토큰을 추가해야 함.
# 한 쌍의 문장이 있는 경우에는 각 문장 뒤에 [SEP]를 붙임. 이를 위해 TemplateProcessor를
# 사용하지만 먼저 vocabulary에서 [CLS] 및 [SEP] 토큰의 ID를 알아야 함
cls_token_id = tokenizer.token_to_id('[CLS]')
sep_token_id = tokenizer.token_to_id('[SEP]')
print(cls_token_id, sep_token_id)

# TemplateProcessor용 템플릿은 단일 문장과 문장 쌍을 처리하는 방법을 지정해야 함.
# 둘 다 사용하려는 특수 토큰을 작성함. 첫 번째(또는 단일) 문장은 $A로 표시되고 
# 두 번째 문장(쌍을 인코딩하는 경우)은 $B로 표시됨. 이들 각각(특수 토큰 및 문장)에 대해 
# 콜론(colon) 뒤에 해당 토큰 유형 ID도 지정함

# 고전적인 BERT 템플릿은 다음과 같이 정의됨
tokenizer.post_processor = processors.TemplateProcessing(
    single=f'[CLS]:0 $A:0 [SEP]:0', pair=f'[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1',
    special_tokens=[("[CLS]", cls_token_id), ("[SEP]", sep_token_id)])

# 특수 토큰의 id를 전달해야 토크나이저가 변환할 수 있음
encoding = tokenizer.encode("Let's test this tokenizer.")
print(encoding.tokens)

encoding = tokenizer.encode(
    "Let's test this tokenizer...", "on a pair of sentences.")
print(encoding.tokens)
print(encoding.type_ids)

2 3
['[CLS]', 'let', "'", 's', 'test', 'this', 'tok', '##eni', '##zer', '.', '[SEP]']
['[CLS]', 'let', "'", 's', 'test', 'this', 'tok', '##eni', '##zer', '...', '[SEP]', 'on', 'a', 'pair', 'of', 'sentences', '.', '[SEP]']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]


In [40]:
# 마지막 디코더 포함 단계
tokenizer.decoder = decoders.WordPiece(prefix='##')
# 테스트
print(tokenizer.decode(encoding.ids))

# tokenizer.save('tokenizer.json') # json 파일 저장 가능
# new_tokenizer = Tokenizer.from_file('tokenizer.json') # Tokenizer 객체에서 로드

# Transformers에서 우리가 만든 토크나이저를 사용하려면 PreTrainedTokenizerFast로 
# 래핑해야 함. 제너릭 클래스(generic class)나, 토크나이저가 기존 모델에 해당하는 경우, 
# 해당 클래스(여기서는 BertTokenizerFast)를 사용할 수 있음.

# PreTrainedTokenizerFast를 가지고 토크나이저를 래핑하려면 우리가 만든 토크나이저를 
# tokenizer_object로 전달하거나 tokenizer_file로 저장한 토크나이저 파일을 전달해야함.
# 중요한 건 모든 특수 토큰을 수동으로 직접 설정해야 한다는 것. 
# PreTrainedTokenizerFast 클래스는 tokenizer 객체로부터 어떤 토큰이 마스크 토큰인지 
# 아니면 [CLS] 토큰 등인지 추론할 수 없기 때문임
from transformers import PreTrainedTokenizerFast
wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer, unk_token='[UNK]', pad_token='[PAD]',
    cls_token='[CLS]', sep_token='[SEP]', mask_token='[MASK]')

# 특정 토크나이저 클래스를 사용하는 경우 지정된 토큰과 다른 특수 토큰만 지정하면 됨
from transformers import BertTokenizerFast
wrapped_tokenizer = BertTokenizerFast(tokenizer_object=tokenizer)

let's test this tokenizer... on a pair of sentences.


In [41]:
# BPE 토크나이저를 처음부터 빌딩하기
# GPT-2 토크나이저 빌드
tokenizer = Tokenizer(models.BPE())

# 앞에서와 마찬가지로 기존에 어휘집(vocabulary)이 있는 경우 모델을 해당 
# 어휘집(vocabulary)으로 초기화할 수 있지만(이 경우 vocab과 merges를 전달해야 함) 
# 처음부터 학습할 것이기 때문에 그럴 필요는 없음.
# 또한 GPT-2는 BPE를 사용하기 때문에 unk_token을 지정할 필요가 없음.

# GPT-2는 노멀라이저를 사용하지 않으므로 사전 토큰화(pre-tokenization)로 바로 이동
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)

# 여기에서 ByteLevel에 추가한 옵션은 문장 시작 부분에 공백을 추가하지 않도록 하는 
# 옵션입니다. 이전과 동일한 사전 토큰화 결과를 아래에서 볼 수 있습니다:
tokenizer.pre_tokenizer.pre_tokenize_str("Let's test pre-tokenization!")
# 다음은 학습이 필요한 모델입니다. GPT-2의 경우, 유일한 특수 토큰은 텍스트 끝(end-of-text) 토큰임
trainer = trainers.BpeTrainer(vocab_size=25000, special_tokens=["<|endoftext|>"])
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

# WordPieceTrainer와 vocab_size 및 special_tokens와 마찬가지로, 원하는 경우 
# min_frequency를 지정할 수 있음. 또는 단어 끝 접미사(end-of-word suffix)가 
# 있는 경우(예: </w>) end_of_word_suffix 인수를 설정할 수 있음
# 이 토크나이저는 텍스트 파일에 대해서도 학습할 수 있음
tokenizer.model = models.BPE()
tokenizer.train(["wikitext-2.txt"], trainer=trainer)

#샘플 텍스트로 토큰화 테스트
encoding = tokenizer.encode("Let's test this tokenizer.")
print(encoding.tokens)

#GPT-2 토크나이저에 대한 바이트 수준(byte-level) 후처리 적용
#trim_offsets = False 옵션은 post-processor가 'Ġ'로 시작하는 토큰의 오프셋을 
# 그대로 두어야 함을 나타냄. 이렇게 하면 오프셋의 시작은 단어의 첫번째 문자가 아니라 
# 단어 앞의 공백을 가리킴. 그 이유는 공백도 기술적으로 토큰의 일부이기 때문.
tokenizer.post_processor = processors.ByteLevel(trim_offsets=False)

# 인코딩한 텍스트 결과.
sentence = "Let's test this tokenizer."
encoding = tokenizer.encode(sentence)

# 'Ġtest'는 인덱스 4의 토큰
start, end = encoding.offsets[4]
sentence[start:end]

# 바이트 수준(byte-level) 디코더 추가
tokenizer.decoder = decoders.ByteLevel()
print(tokenizer.decode(encoding.ids))

# 토크나이저를 저장하고, 이를 Transformers에서 사용하기를 원하는 경우에는 
# PreTrainedTokenizerFast 또는 GPT2TokenizerFast를 이용하여 래핑(wrapping)할 수 있음
from transformers import PreTrainedTokenizerFast
wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer, bos_token="<|endoftext|>", eos_token="<|endoftext|>",)

from transformers import GPT2TokenizerFast
wrapped_tokenizer = GPT2TokenizerFast(tokenizer_object=tokenizer)







['L', 'et', "'", 's', 'Ġtest', 'Ġthis', 'Ġto', 'ken', 'izer', '.']
Let's test this tokenizer.


In [42]:
# Unigram 토크나이저를 처음부터 빌딩하기
# XLNet 토크나이저 빌드. Unigram 모델로 Tokenizer 초기화
tokenizer = Tokenizer(models.Unigram())

# 어휘집(vocabulary)이 있는 경우 모델을 어휘집으로 초기화할 수 있음
# 정규화(normalization)를 위해 XLNet은 몇 가지 대체규칙(relpacements, 
# SentencePiece에서 제공)을 사용함
from tokenizers import Regex
tokenizer.normalizer = normalizers.Sequence([
        normalizers.Replace("``", '"'), normalizers.Replace("''", '"'),
        normalizers.NFKD(), normalizers.StripAccents(),
        normalizers.Replace(Regex(" {2,}"), " "),])

# 위 대체규칙은 “ 및 ”를 ”로 대체하고, 둘 이상의 공백 시퀀스를 단일 공백으로 대체하며, 
# 토큰화할 텍스트의 악센트를 제거함
# 모든 SentencePiece 토크나이저에 사용되는 
# 사전 토크나이저(pre-tokenizer)는 Metaspace 임.
tokenizer.pre_tokenizer = pre_tokenizers.Metaspace()

# 이전 사전 토큰화 결과
tokenizer.pre_tokenizer.pre_tokenize_str("Let's test the pre-tokenizer!")

# 모델 학습. XLNet의 몇 가지 특별한 토큰
special_tokens = ["<cls>", "<sep>", "<unk>", "<pad>", "<mask>", "<s>", "</s>"]
trainer = trainers.UnigramTrainer(
    vocab_size=25000, special_tokens=special_tokens, unk_token="<unk>")
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

# 중요한 인수는 unk_token. 또한 토큰을 제거하는 각 단계에 대한 
# shrinking_factor(기본값은 0.75), 또는 주어진 토큰의 최대 길이를 지정하기 위한 
# max_piece_length(기본값은 16) 등과 같은 Unigram 알고리즘에 특화된 
# 추가 매개변수들을 전달할 수도 있음

# 이 토크나이저는 역시 텍스트 파일에 대해서도 학습할 수 있음
tokenizer.model = models.Unigram()
tokenizer.train(["wikitext-2.txt"], trainer=trainer)

# 샘플 텍스트의 토큰화 결과
encoding = tokenizer.encode("Let's test this tokenizer.")
print(encoding.tokens)

# XLNet의 특징은 토큰 타입 ID가 2인 <cls> 토큰을 문장 끝에 추가한다는 것.
# 결과적으로 좌측 패딩. BERT와 같이 템플릿을 사용하여 모든 특수 토큰과 
# 토큰 타입 ID를 처리할 수 있지만 그전에 먼저 <cls> 및 <sep> 토큰의 ID를 가져와야 함
cls_token_id = tokenizer.token_to_id("<cls>")
sep_token_id = tokenizer.token_to_id("<sep>")
print(cls_token_id, sep_token_id)

# 템플릿 구현
tokenizer.post_processor = processors.TemplateProcessing(
    single="$A:0 <sep>:0 <cls>:2", pair="$A:0 <sep>:0 $B:1 <sep>:1 <cls>:2",
    special_tokens=[("<sep>", sep_token_id), ("<cls>", cls_token_id)],)

# 한 쌍의 문장에 대한 인코딩 테스트
encoding = tokenizer.encode("Let's test this tokenizer...", "on a pair of sentences!")
print(encoding.tokens)
print(encoding.type_ids)

# Metaspace 디코더 추가
tokenizer.decoder = decoders.Metaspace()

# 토크나이저를 저장하고 Transformers 내에서 사용하려면 PreTrainedTokenizerFast 또는 
# XLNetTokenizerFast로 래핑할 수 있음. PreTrainedTokenizerFast를 사용할 때 
# 주의해야 할 점은 특수 토큰 지정과 더불어 Transformers 라이브러리에 
# padding_side="left"를 지정해야 한다는 것임
from transformers import PreTrainedTokenizerFast
wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    bos_token="<s>", eos_token="</s>", unk_token="<unk>", pad_token="<pad>",
    cls_token="<cls>", sep_token="<sep>", mask_token="<mask>", padding_side="left",)

from transformers import XLNetTokenizerFast
wrapped_tokenizer = XLNetTokenizerFast(tokenizer_object=tokenizer)





['▁Let', "'", 's', '▁test', '▁this', '▁to', 'ken', 'izer', '.']
0 1
['▁Let', "'", 's', '▁test', '▁this', '▁to', 'ken', 'izer', '.', '.', '.', '<sep>', '▁', 'on', '▁', 'a', '▁pair', '▁of', '▁sentence', 's', '!', '<sep>', '<cls>']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2]
